## 目的：利用MNE实现自定义矩阵大脑拓扑图的绘制

### 0、加载python库

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import mne

%matplotlib
plt.show()

Using matplotlib backend: MacOSX


### 1、获取可用的电极布局系统

In [3]:
mne.channels.get_builtin_montages()

['standard_1005',
 'standard_1020',
 'standard_alphabetic',
 'standard_postfixed',
 'standard_prefixed',
 'standard_primed',
 'biosemi16',
 'biosemi32',
 'biosemi64',
 'biosemi128',
 'biosemi160',
 'biosemi256',
 'easycap-M1',
 'easycap-M10',
 'EGI_256',
 'GSN-HydroCel-32',
 'GSN-HydroCel-64_1.0',
 'GSN-HydroCel-65_1.0',
 'GSN-HydroCel-128',
 'GSN-HydroCel-129',
 'GSN-HydroCel-256',
 'GSN-HydroCel-257',
 'mgh60',
 'mgh70',
 'artinis-octamon',
 'artinis-brite23',
 'brainproducts-RNP-BA-128']

### 2、利用MNE自带的电极布局系统对矩阵进行通道定位

#### 2.1 加载脑地形图位置坐标并可视化

In [4]:
int32_montage = mne.channels.make_standard_montage('biosemi32')
int32_montage.plot()
plt.show()

#### 2.2 构建自定义32导联的矩阵

In [5]:
weight = {'Fp1': 0.31, 'AF3': -0.26, 'F7': 1.22, 'F3': 0.99, 'Fz': 0.71, 'FC5': 0.55,
          'FC1': -1.19, 'T7': 0.61, 'C3': -0.80, 'Cz': 2.36, 'CP5': -0.74, 'CP1': 0.72,
          'P7': 0.93, 'P3': 0.38, 'Pz': 1.07, 'PO3': -1.46, 'O1': -0.12, 'Oz': 1.074,
          'Fp2': 1.04, 'AF4': -0.065, 'F4': -0.52, 'F8': 0.37, 'FC2': 1.30, 'FC6': 0.94,
          'C4': -1.11, 'T8': -0.16, 'CP2': 1.82, 'CP6': 0.41, 'P4': 0.46, 'P8': 0.99, 'PO4':0.15, 'O2':0.23}

#### 2.3 根据脑地形图导联顺序重构自定义矩阵（*注意）

In [6]:
# 查看脑地形图矩阵导联位置
sensor_data_32 = int32_montage.get_positions()['ch_pos']
sensor_dataframe_32 = pd.DataFrame(sensor_data_32).T
chLa_index = sensor_dataframe_32.index.values
print('脑地形图矩阵导联顺序:',chLa_index)

# 重构自定义矩阵顺序
reWeight = []
for key in chLa_index:
    val = weight[key]
    reWeight.append(val)

脑地形图矩阵导联顺序: ['Fp1' 'AF3' 'F7' 'F3' 'FC1' 'FC5' 'T7' 'C3' 'CP1' 'CP5' 'P7' 'P3' 'Pz'
 'PO3' 'O1' 'Oz' 'O2' 'PO4' 'P4' 'P8' 'CP6' 'CP2' 'C4' 'T8' 'FC6' 'FC2'
 'F4' 'F8' 'AF4' 'Fp2' 'Fz' 'Cz']


### 3、TopoMap可视化

#### 3.1 创建info

In [7]:
info = mne.create_info(
        ch_names = chLa_index.tolist(),
        ch_types = ['eeg']*32,   # 通道个数
        sfreq = 1000)            # 采样频率
info.set_montage(int32_montage)

<Info | 8 non-empty values
 bads: []
 ch_names: Fp1, AF3, F7, F3, FC1, FC5, T7, C3, CP1, CP5, P7, P3, Pz, PO3, ...
 chs: 32 EEG
 custom_ref_applied: False
 dig: 35 items (3 Cardinal, 32 EEG)
 highpass: 0.0 Hz
 lowpass: 500.0 Hz
 meas_date: unspecified
 nchan: 32
 projs: []
 sfreq: 1000.0 Hz
>

#### 3.2 可视化

In [8]:
im, cn = mne.viz.plot_topomap(reWeight, 
                              info, 
                              names = chLa_index.tolist(),
                              # vlim=(-2, 2)
                             )
plt.colorbar(im)
plt.show()

### 4、自定义电极布局系统对矩阵进行通道定位
假设我有一个64导联的帽子，根据实验需求，实验过程中我只选择中央区和顶叶区域的电极。  

那么我们该如何根据自己选择的导联制定通道定位模版呢？

#### 4.1 下载标准的64通道电极布局系统坐标

In [9]:
# 读取MNE中biosemi64电极位置信息
biosemi_montage = mne.channels.make_standard_montage('biosemi64')
sensor_data_64 = biosemi_montage.get_positions()['ch_pos']
sensor_dataframe_64 = pd.DataFrame(sensor_data_64).T
sensor_dataframe_64.to_excel('sensor_dataframe_64.xlsx')

#### 4.2 根据下载文件，选择实验选取的电极坐标，保存为excel格式

In [10]:
# 模仿下载的电极布局excel，自定义电极布局文件

#### 4.3 加载自定义的excel文件，制作自己的montage

In [11]:
myStardart = pd.read_excel('mySensor_dataframe.xlsx', index_col=0) # 读取自己的电极定位文件
ch_names = np.array(myStardart.index)                              # 电极名称
position = np.array(myStardart)                                    # 电极坐标位置
sensorPosition = dict(zip(ch_names, position))                     # 制定为字典的形式
myMontage = mne.channels.make_dig_montage(ch_pos=sensorPosition)
myMontage.plot()
plt.show()

/var/folders/fj/lgd84tbn0vlg7xj60s05pkph0000gn/T/ipykernel_4947/2900886980.py:6: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  myMontage.plot()


#### 4.4 自定义导联权重矩阵

In [12]:
myWeight = {'C1': 0.31, 'C3': 1.22, 'P6': 0.71, 'P4': 0.55,'P2': -1.19, 'CP5': 0.61,
            'CP3': -0.80, 'CP1': 2.36, 'P1': -0.74, 'P3': 0.72,'P5': 0.93, 'Pz': -1.46, 
            'CPz': -0.12, 'Cz': 1.074,'C5':2.34,'C2': 1.04, 'C4': -0.065, 'C6': -0.52, 
            'CP2': 0.37, 'CP4': 1.30, 'CP6': 0.94}

#### 4.5 根据脑地形图导联顺序重构自定义矩阵（*注意）

In [13]:
# 查看脑地形图矩阵导联位置
my_chLa_index = ch_names.tolist()
print('脑地形图矩阵导联顺序:',my_chLa_index)

# 重构自定义矩阵顺序
reMyWeight = []
for key in my_chLa_index:
    val = myWeight[key]
    reMyWeight.append(val)

脑地形图矩阵导联顺序: ['C1', 'C3', 'C5', 'CP5', 'CP3', 'CP1', 'P1', 'P3', 'P5', 'Pz', 'CPz', 'Cz', 'C2', 'C4', 'C6', 'CP6', 'CP4', 'CP2', 'P2', 'P4', 'P6']


### 5、TopoMap可视化

#### 5.1 创建info

In [14]:
myinfo = mne.create_info(
        ch_names = my_chLa_index,
        ch_types = ['eeg']*21,   # 通道个数
        sfreq = 1000)            # 采样频率
myinfo.set_montage(myMontage)

/var/folders/fj/lgd84tbn0vlg7xj60s05pkph0000gn/T/ipykernel_4947/778578395.py:5: RuntimeWarning: Fiducial point nasion not found, assuming identity unknown to head transformation
  myinfo.set_montage(myMontage)


<Info | 8 non-empty values
 bads: []
 ch_names: C1, C3, C5, CP5, CP3, CP1, P1, P3, P5, Pz, CPz, Cz, C2, C4, C6, ...
 chs: 21 EEG
 custom_ref_applied: False
 dig: 24 items (3 Cardinal, 21 EEG)
 highpass: 0.0 Hz
 lowpass: 500.0 Hz
 meas_date: unspecified
 nchan: 21
 projs: []
 sfreq: 1000.0 Hz
>

#### 5.2 可视化

In [15]:
im, cn = mne.viz.plot_topomap(reMyWeight, 
                              myinfo, 
                              names = my_chLa_index,
                              # vlim=(-2, 2)
                             )
plt.colorbar(im)
plt.title('My Montage')
plt.show()